# Wyszukiwanie pełnotekstowe w PostgreSQL

Używam tego jako jednego z wariantów komponentu retrieval w moim RAGu

Najpierw załadowałem bazę skryptem `just seed`

In [ ]:
%%sql
DROP INDEX IF EXISTS content_idx;

In [1]:
%%sql
--- Wyszukiwanie bez indeksu na podstawie pełnotekstowego zapytania
SELECT note_chunks.title, note_chunks.filename, note_chunks.course
FROM note_chunks
WHERE to_tsvector(note_chunks.content) @@ to_tsquery('transformata & fouriera');


,title,filename,course
0,Szereg i transformata Fouriera,02-szereg-i-transformata-fouriera,wstep-do-multimediow
1,Sygnały akustyczne,15-sygnaly-akustyczne,wstep-do-multimediow
2,Filtracja cyfrowa,05-filtracja-cyfrowa,wstep-do-multimediow


In [2]:
%%sql
--- Wyszukiwanie bez indeksu, zapytanie zbudowane ze zwykłego tekstu
SELECT note_chunks.title, note_chunks.filename, note_chunks.course
FROM note_chunks
WHERE to_tsvector(note_chunks.content) @@ phraseto_tsquery('transformata fouriera');


,title,filename,course
0,Szereg i transformata Fouriera,02-szereg-i-transformata-fouriera,wstep-do-multimediow
1,Sygnały akustyczne,15-sygnaly-akustyczne,wstep-do-multimediow
2,Filtracja cyfrowa,05-filtracja-cyfrowa,wstep-do-multimediow


In [3]:
%%sql
--- Plan wykonania wyszukiwania bez indeksu
EXPLAIN ANALYSE
SELECT note_chunks.title, note_chunks.filename, note_chunks.course
FROM note_chunks
WHERE to_tsvector(note_chunks.content) @@ phraseto_tsquery('transformata fouriera');


,QUERY PLAN
0,Seq Scan on note_chunks (cost=0.00..2581.84 r...
1,Filter: (to_tsvector(content) @@ phraseto_ts...
2,Rows Removed by Filter: 4344
3,Planning Time: 1.700 ms
4,Execution Time: 422.523 ms


In [4]:
%%sql
--- Out of the box nie ma wsparcia dla języka polskiego, ale można dodać
SELECT cfgname FROM pg_ts_config;

,cfgname
0,simple
1,arabic
2,armenian
3,basque
4,catalan
5,danish
6,dutch
7,english
8,finnish
9,french


In [5]:
%%sql
SHOW default_text_search_config;

,default_text_search_config
0,pg_catalog.english


In [ ]:
%%sql
--- Utworzenie indeksu
CREATE INDEX content_idx ON note_chunks USING GIN (to_tsvector('simple', content));
-- musi być podana konfiguracja

In [19]:
%%sql
--- Wyszukiwanie z indeksem
-- jest troszkę szybsze, mamy mało danych
EXPLAIN ANALYSE
SELECT note_chunks.title, note_chunks.filename, note_chunks.course
FROM note_chunks
WHERE to_tsvector('simple', note_chunks.content) @@ phraseto_tsquery('simple', 'transformata fouriera');


,QUERY PLAN
0,Bitmap Heap Scan on note_chunks (cost=21.54.....
1,Recheck Cond: (to_tsvector('simple'::regconf...
2,Heap Blocks: exact=3
3,-> Bitmap Index Scan on content_idx (cost=...
4,Index Cond: (to_tsvector('simple'::reg...
5,Planning Time: 0.570 ms
6,Execution Time: 0.308 ms


In [17]:
%%sql
-- musi być podana ta sama konfiguracja co w indeksie, żeby go użyć
EXPLAIN ANALYSE
SELECT note_chunks.title, note_chunks.filename, note_chunks.course
FROM note_chunks
WHERE to_tsvector(note_chunks.content) @@ phraseto_tsquery('transformata fouriera');


,QUERY PLAN
0,Seq Scan on note_chunks (cost=0.00..2581.84 r...
1,Filter: (to_tsvector(content) @@ phraseto_ts...
2,Rows Removed by Filter: 4344
3,Planning Time: 1.746 ms
4,Execution Time: 323.257 ms


## Ranking wyników wyszukiwania

In [23]:
%%sql
--EXPLAIN ANALYSE
SELECT note_chunks.title, note_chunks.filename, note_chunks.course, ts_rank(to_tsvector('simple', note_chunks.content), phraseto_tsquery('simple', 'solid')) as rank
FROM note_chunks
WHERE to_tsvector('simple', note_chunks.content) @@ phraseto_tsquery('simple', 'solid')
ORDER BY rank DESC
LIMIT 10;

,title,filename,course,rank
0,Warsztat inżyniera oprogramowania (2024-02-26),wyklad-02,sztuka-wytwarzania-oprogramowania,0.075991
1,Wstęp do inżynierii oprogramowania,09-wstep-do-inzynierii-oprogramowania,podstawy-informatyki-i-programowania,0.060793
2,Projektowanie i programowanie (2024-03-18),wyklad-05,sztuka-wytwarzania-oprogramowania,0.060793
3,Kolokwium 01,kolokwium-01,sztuka-wytwarzania-oprogramowania,0.060793
4,Warsztat inżyniera oprogramowania (2024-02-26),wyklad-02,sztuka-wytwarzania-oprogramowania,0.060793
5,Refaktoring (2024-05-13),wyklad-10,sztuka-wytwarzania-oprogramowania,0.060793
6,Grafika komputerowa,22-grafika-komputerowa,wstep-do-multimediow,0.060793
7,Wstęp do inżynierii oprogramowania,09-wstep-do-inzynierii-oprogramowania,podstawy-informatyki-i-programowania,0.060793
8,Reprezentacje obiektów,01-reprezentacje-obiektow,grafika-komputerowa,0.060793


## Wsparcie języka polskiego

Korzystając z https://github.com/judehunter/polish-tsearch dodałem konfigurację dla języka polskiego

In [4]:
%%sql
SELECT websearch_to_tsquery('polish', 'Wytłumacz mi jak działa transformata Fouriera');

,websearch_to_tsquery
0,'wytłumaczyć' & 'jaka' & ( 'działa' | 'działać...


In [5]:
%%sql
SELECT
    id, title, course,
    websearch_to_tsquery('polish', 'Jak działa semafor?')
FROM note_chunks
WHERE to_tsvector('polish', content) @@ websearch_to_tsquery('polish', 'Jak działa semafor?');

,id,title,course,websearch_to_tsquery
0,b000cba0-081b-483a-b3b4-63e6399ef8a0,wywołanie z 3 argumentami,systemy-operacyjne,'jaka' & ( 'działa' | 'działać' | 'dziać' ) & ...
1,47447adc-b02b-40d0-9b41-9772442c5986,wywołanie z 3 argumentami,systemy-operacyjne,'jaka' & ( 'działa' | 'działać' | 'dziać' ) & ...


## Użycie w ORM

In [1]:
import os

os.chdir("..")

In [2]:
from notes_rag.core.schema import (
    get_engine,
    NoteChunk,
    DbConfig,
    create_session_factory,
    NoteChunkRepository,
)
from sqlalchemy import select, func
from sqlalchemy.dialects import postgresql
from sqlalchemy.orm import Session


def search_notes(query: str, session):
    stmt = select(
        NoteChunk.title,
        NoteChunk.filename,
        NoteChunk.course,
    ).where(
        func.to_tsvector("simple", NoteChunk.content).op("@@")(
            func.phraseto_tsquery("simple", query)
        )
    )

    print(stmt.compile(dialect=postgresql.dialect()))

    return session.execute(stmt).all()


eng = get_engine(DbConfig.local())
with eng.connect() as conn:
    with Session(conn) as session:
        results = search_notes("transformata fouriera", session)
        for row in results:
            print(row)

SELECT note_chunks.title, note_chunks.filename, note_chunks.course 
FROM note_chunks 
WHERE to_tsvector(%(to_tsvector_1)s, note_chunks.content) @@ phraseto_tsquery(%(phraseto_tsquery_1)s, %(phraseto_tsquery_2)s)
('Szereg i transformata Fouriera', '02-szereg-i-transformata-fouriera', 'wstep-do-multimediow')
('Sygnały akustyczne', '15-sygnaly-akustyczne', 'wstep-do-multimediow')
('Filtracja cyfrowa', '05-filtracja-cyfrowa', 'wstep-do-multimediow')


## Użycie kodu biblioteki

In [4]:
eng = get_engine(DbConfig.local())
sm = create_session_factory(eng)
with sm() as session:
    repo = NoteChunkRepository(session)
    result = repo.search_fulltext("solid", limit=3)

    for row in result:
        print(row)

NoteChunk(id=581e6f2d-9b84-4f5a-a88f-4c0cc36586c4, title=Warsztat inżyniera oprogramowania (2024-02-26), course=sztuka-wytwarzania-oprogramowania, number=4)
NoteChunk(id=0e40bcb1-1b1e-4d49-b3bc-28b6a95d9dd8, title=Projektowanie i programowanie (2024-03-18), course=sztuka-wytwarzania-oprogramowania, number=0)
NoteChunk(id=44225168-8872-4ecf-8db3-74f2db46b8ea, title=Wstęp do inżynierii oprogramowania, course=podstawy-informatyki-i-programowania, number=12)
